In [1]:
# Manually override these values for ad hoc use.

# TAEGIS_TENANT_ID = ""
# TAEGIS_ENVIRONMENT = ""
# INVESTIGATION_TITLE = ""

> This is a trivial procedure intended for use in Tutorial 4 - Hunting at Scale.
> The content of this notebook is presented for demonstration purposes only.

# Suspicious Windows Service Manipulation

## Executive Summary

The goal of this procedure is to identify possible anti-forensic and defense evasion techniques related to Windows service manipulation.
The Windows operating system ensures that enabled services are running.
Many security controls and monitoring software run as Windows services.
Threat actors often disable Windows services associated with security controls and monitoring software, such as anti-virus and EDR software, to avoid detection and impede forensic investigation.

## Procedures

On Windows, the native binary `sc.exe` is used to configure Windows services.
It is uncommon for users or administrators to run `sc.exe` since Windows services on these systems are centrally managed by group policy.
We will review all invocations of `sc.exe` to look for unusual users and command-line arguments suggestive of disabling known security tools. 

In [2]:
import pandas as pd
from IPython.display import display_markdown

def md(markdown_text: str):
    return display_markdown(markdown_text, raw=True)

%load_ext taegis_magic

### Step 1: Review Recent Service-Related Alerts

Please review all recent alerts related to Windows service manipulation that were lower than high severity.

In [ ]:
%%taegis alerts search --tenant $TAEGIS_TENANT_ID --region $TAEGIS_ENVIRONMENT --track --cache --assign sc_alerts

FROM alert
WHERE 
    metadata.severity <= 0.6 AND
    (metadata.title contains 'service' OR metadata.title contains 'sc.exe')
EARLIEST='2023-09-12T12:47:00'
LATEST='2023-09-12T13:05:00'

In [ ]:
if sc_alerts.empty:
    md("No recent alerts were identified related to the manipulation of Windows services.")
else:
    display(sc_alerts.groupby(["status", "metadata.severity", "metadata.title"])["id"].nunique().to_frame())

In [ ]:
# Intentionally left commented out!
# Analysts are encouraged to uncomment when ready to attach evidence.
#%taegis investigations evidence stage alerts sc_alerts

### Step 2: Enumerate `sc.exe` Processes

In [ ]:
%%taegis events search --tenant $TAEGIS_TENANT_ID --region $TAEGIS_ENVIRONMENT --track --cache --assign sc_processes

FROM process
WHERE image_path contains 'sc.exe'
EARLIEST='2023-09-12T12:47:00'
LATEST='2023-09-12T13:05:00'

### Step 3: Find Unusual User Execution of `sc.exe`

In our environment, invocations of `sc.exe` should be performed by administrator users or the `SYSTEM` account.
Administrator usernames must start with `adm_*`.
Please review all invocations of `sc.exe` performed by unauthorized or unknown user accounts.

In [ ]:
step_3_evidence = sc_processes[~sc_processes.username.str.contains("adm_|SYSTEM")]
if step_3_evidence.empty:
    md("`sc.exe` was not executed by non-SYSTEM, non-administrator users.")
else:
    display(step_3_evidence.groupby(["username", "parent_image_path", "image_path", "commandline"]).size().to_frame())

In [ ]:
# Intentionally left commented out!
# Analysts are encouraged to uncomment when ready to attach evidence.
#%taegis investigations evidence stage events step_3_evidence

### Step 4: Identify Anomalous Command-line Arguments

The `sc.exe` command takes several arguments that indicate which service configurations should be changed.
Since Windows service configuration in our environment is managed by Group Policy, manual `sc stop` and `sc disable` commands are not expected behavior.
`sc stop` and `sc disable` commands are more suspicious if the target service is associated with a security control or monitoring software.
Please review these `sc` commands for unauthorized service manipulation.

In [ ]:
step_4_evidence = sc_processes[sc_processes.commandline.str.contains("stop|delete", case=False)]
if step_4_evidence.empty:
    md("No `sc stop` or `sc disable` commands were observed.")
else:
    display(step_4_evidence)

In [ ]:
# Intentionally left commented out!
# Analysts are encouraged to uncomment when ready to attach evidence.
#%taegis investigations evidence stage events step_4_evidence

## Findings

*This should be filled in by the analyst when using the notebook.*

## Recommendations

If suspicious service manipulation was discovered, please follow incident response playbook XYZ.

**Please remember to save the notebook and set the cell type to "Code" before running the following cell.**